# Quant 101
## #2 Régression Linéaire : $\alpha$, $\beta$ et le CAPM

## Rappel de l'atelier précédent
- Rendements : simples vs logarithmiques. Comment le prix d'un actif évolue-t-il dans le temps ?
- Volatilité : mesure du risque. Comment quantifier les fluctuations d'un actif ?
- Corrélations : interdépendance entre actifs. Comment les actifs évoluent-ils ensemble ?

### Lien avec la régression linéaire
- En finance, on utilise souvent la régression linéaire pour estimer le **$\alpha$** (rendement excédentaire) et le **$\beta$** (sensibilité au marché) d'un actif.
- On passe d'une relation descriptive (corrélation) à une relation quantifiée (régression).
    - Après avoir appris à mesurer, on apprend à expliquer et prédire.

## Objectifs du jour
- Comprendre et pouvoir estimer les valeurs d'**$\alpha$** et de **$\beta$**
- Introduire le concept du **CAPM** (Capital Asset Pricing Model) de manière pratique
- Construire un mini-modèle de pricing sur des données réelles

## Intuition derrière le modèle de régression linéaire

*Correction : les figures suivantes sont en fonction de rendements (non de prix!)

![Scatter](figures/scatter.png)

### Pente 
**$\beta$** : mesure la sensibilité de l'actif aux mouvements du marché.
- **$\beta = 1$** : l'actif suit parfaitement le marché.
- **$\beta > 1$** : l'actif amplifie les mouvements du marché.
- **$\beta < 1$** : l'actif amortit les mouvements du marché.
- **$\beta < 0$** : l'actif évolue en sens inverse du marché.

- Tendance moyenne extraite via la régression linéaire.

### Ordonnée à l'origine 
**$\alpha$** : rendement excédentaire qui s'ajoute à celui prédit par le marché.
- **$\alpha > 0$** : l'actif surperforme par rapport à ce que le marché suggère.
- **$\alpha < 0$** : l'actif sous-performe par rapport à ce que le marché suggère.

- La partie de la performance non expliquée par le marché.

### La dispersion (variance des résidus) 
Le risque spécifique à l'actif, non capturé par le marché.
- Plus la dispersion est grande, plus l'actif est risqué indépendamment des mouvements du marché.
- Quantifié par le R² de la régression (haut coeff $\rightarrow$ faible dispersion).
- Lié au concept de risque *spécifique* (idiosyncratique)

### Risque systématique vs spécifique
#### Risque systématique
Le risque provenant du *marché lui-même* : cycles économiques, sentiment global, événements macroéconomiques.
- Affecte tous les actifs.
- Ne peut pas être éliminé par la diversification.
- Capturé par le **$\beta$**.

#### Risque spécifique (idiosyncratique)
Le risque propre à un actif individuel : management, structure financière, événements propres à l'entreprise.
- Peut être réduit par la diversification.
- Capturé par la dispersion des résidus dans la régression.


![Lin reg](figures/linreg.png)


## Statistiques de base

On tente de fit une droite aux points (rendements de l'actif vs rendements du marché) en minimisant la somme des carrés des distances verticales (résidus).  
On a un modèle de la forme : 
$$
r_i = \alpha + \beta r_m + \epsilon
$$

### Définitions
- $\beta$ = Cov($r_i$, $r_m$) / Var($r_m$)
- $\alpha$ = intercept $\rightarrow$ non-expliqué par le marché
- $\epsilon$ = bruit $\rightarrow$ diversifiable
- R² = proportion de la variance (du risque) expliquée par le modèle

### p-value
- Mesure la signification statistique des coefficients estimés.
- p-value faible (généralement < 0.05) indique que le coefficient est statistiquement significatif.
- Permet de tester l'hypothèse nulle selon laquelle le coefficien n'a pas d'effet